In [ ]:
import numpy as np 
import pandas as pd
from pandas.api.types import CategoricalDtype

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

pd.set_option('max_column', 30)

In [ ]:
def newline(ax, p1, p2, color='black'):
    l = mlines.Line2D([p1[1],p2[1]], [p1[0],p2[0]], color=color)
    ax.add_line(l)
    return ax

def plot_frame(ax):
    ax.set_facecolor('#292525')
    ax.spines['bottom'].set_color('w')
    ax.tick_params(axis='x', colors='w')
    ax.xaxis.label.set_color('w')
    ax.spines['left'].set_color('w')
    ax.tick_params(axis='y', colors='w')
    ax.yaxis.label.set_color('w')
    return ax

def annotate_bar(ax, data):
    rects = ax.patches
    labels = [f'n = {count}' for count in data['count']]
    if data['mean'].max() > 1000:
        x_pos = 1000
    else:
        x_pos = 0.1
    for rect, label in zip(rects, labels):
        y_value = rect.get_y() + rect.get_height() / 2
        ax.text(x_pos, y_value, label, color='w', va='center')
    return ax


def plot_means(data, target, question, title, sub_title, target_title):
    fig = plt.figure(figsize=(15, 14), facecolor='#292525')
    fig.subplots_adjust(top=0.94)
    fig.suptitle(title, fontsize=18, color='w')

    gs = GridSpec(2, 9, figure=fig)
    ax0 = fig.add_subplot(gs[0, :4])
    ax1 = fig.add_subplot(gs[0, 5:])
    ax2 = fig.add_subplot(gs[1, :])

    tmp = data.groupby(question)[target].agg(['mean', 'count'])
    tmp['mean'].plot(kind='barh', ax=ax0, color='#3A3FDC')
    ax0.axvline(data[target].mean(), color='w', linestyle='--')
    ax0 = annotate_bar(ax0, tmp)
    ax0.set_title(f'Mean {target_title} by {sub_title}', color='w', fontsize=14)
    
    tmp = data.groupby('Q2')[target].agg(['mean', 'count'])
    tmp['mean'].plot(kind='barh', ax=ax1, color='#3A3FDC')
    ax1.axvline(data[target].mean(), color='w', linestyle='--')
    ax1 = annotate_bar(ax1, tmp)
    ax1.set_title(f'Mean {target_title} by Gender', color='w', fontsize=14)

    tmp = data.groupby([question, 'Q2'])[target].mean().unstack().reset_index()

    counts = data.groupby([question, 'Q2']).size().unstack()

    ax2.scatter(y=tmp['Man'], x=tmp[question], color='#C3C92E', s=100, alpha=0.8, label='Men')
    ax2.scatter(y=tmp['Woman'], x=tmp[question], color='#C93D2E', s=100, alpha=0.8, label='Women')
    ax2.scatter(y=tmp['Other'], x=tmp[question], color='#41DA5B', s=100, alpha=0.8, label='Other')

    ax2.axhline(data[data.Q2 == 'Man'][target].mean(), color='#C3C92E', linestyle='--', alpha=0.5)
    ax2.axhline(data[data.Q2 == 'Woman'][target].mean(), color='#C93D2E', linestyle='--', alpha=0.5)
    ax2.axhline(data[data.Q2 == 'Other'][target].mean(), color='#41DA5B', linestyle='--', alpha=0.5)
    legend = ax2.legend(facecolor='#292525', edgecolor='#292525')
    plt.setp(legend.get_texts(), color='w')
    ax2.set_title(f'Mean {target_title} by {sub_title} and Gender', color='w', fontsize=14)
    plt.draw()
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=35)
    

    for i, p1, p2 in zip(tmp[question], tmp['Man'], tmp['Woman']):
            ax2 = newline(ax2, [p1, i], [p2, i], color='silver')
    for i, p1, p2 in zip(tmp[question], tmp['Man'], tmp['Other']):
            ax2 = newline(ax2, [p1, i], [p2, i], color='silver')
            

    for ax in [ax0, ax1, ax2]:
        ax = plot_frame(ax)
        ax.set_xlabel('')
        ax.set_ylabel('')
        
    ax2.set_ylabel(f'Mean {target_title}', color='w', fontsize=10)

    plt.show()

In [ ]:
data = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv', low_memory=False)
questions = data.iloc[0, :].T
data = data.iloc[1:, :]

def total_question(data, question):
    qst_lst = [col for col in data if question+'_' in col]
    data[question+'_total'] = len(qst_lst) - data[qst_lst].isna().sum(axis=1)
    
    return data

def clean_data(data):
    data['Q4'] = data['Q4'].replace({"Some college/university study without earning a bachelor’s degree": 'College, no degree', 
                    'No formal education past high school': 'No formal education', 
                                 'I prefer not to answer': 'No Answer'}).fillna('No Answer')
    cat_type = CategoricalDtype(categories=["No Answer", "No formal education", "Professional degree", 
                                            "College, no degree", "Bachelor’s degree", 
                                            "Master’s degree", "Doctoral degree"], ordered=True)
    data['Q4'] = data['Q4'].astype(cat_type)

    income_dict = {'$0-999': 500, '10,000-14,999': 12500, '1,000-1,999': 1500, '100,000-124,999': 112500, 
                   '40,000-49,999': 45000, '30,000-39,999': 35000, '50,000-59,999': 55000, '5,000-7,499': 6250, 
                   '15,000-19,999': 17500, '60,000-69,999': 65000, '20,000-24,999': 22500, '70,000-79,999': 75000, 
                   '7,500-9,999': 8250, '150,000-199,999': 175000, '2,000-2,999': 2500, '125,000-149,999': 137500, 
                   '25,000-29,999': 27500, '90,000-99,999': 95000, '4,000-4,999': 4500, '80,000-89,999': 85000, 
                   '3,000-3,999': 3500, '200,000-249,999': 225000, '300,000-500,000': 400000, '> $500,000': 500000, 
                   '250,000-299,999': 275000}

    data['Q24_num'] = pd.to_numeric((data['Q24'].map(income_dict))) # .fillna(0)

    data['Q6'] = data['Q6'].replace({'I have never written code': '0 years'}).fillna('0 years')
    exp_cats = CategoricalDtype(categories=["0 years", "< 1 years", "1-2 years",
                                            "3-5 years", "5-10 years", 
                                            "10-20 years", "20+ years"], ordered=True)
    data['Q6'] = data['Q6'].astype(exp_cats)

    # for better merge with the continents
    data['Q3'] = data['Q3'].replace({'Iran, Islamic Republic of...': 'Iran (Islamic Republic of)', 
                                     'Russia': 'Russian Federation', 
                                     'Taiwan': 'Taiwan, Province of China', 
                                     'South Korea': 'Korea (Republic of)', 
                                     'Republic of Korea': 'Korea (Republic of)'})

    continents = pd.read_csv('/kaggle/input/country-to-continent/countryContinent.csv', encoding = 'ISO-8859-1')
    data = pd.merge(data, continents[['country', 'continent', 'sub_region']], left_on='Q3', right_on='country', how='left')


    data['Q3'] = data['Q3'].replace({'United States of America': 'USA', 
                                     'United Kingdom of Great Britain and Northern Ireland': 'UK', 
                                     'Russian Federation': 'Russia', 
                                     'Korea (Republic of)': 'S. Korea', 
                                     'Taiwan, Province of China': 'Taiwan'})

    data['sub_region'] = (data.sub_region.str.replace('ern', '')
                          .str.replace('South', 'S')
                          .str.replace('North', 'N')
                          .str.replace('West', 'W')
                          .str.replace('East', 'E')
                          .str.replace('Central', 'C')
                          .str.replace('Australia and New Zealand', 'Aus and NZ'))
    sub_reg = CategoricalDtype(categories=["E Africa", "N Africa", "S Africa", "W Africa",
                                            "C America", "N America", "S America",
                                            "E Asia", "S-E Asia", "S Asia", "W Asia", "Aus and NZ",
                                           "E Europe", "N Europe", "S Europe", "W Europe"], ordered=True)
    data['sub_region'] = data['sub_region'].astype(sub_reg)

    exp_cats = CategoricalDtype(categories=["0 years", "Under 1 year", "1-2 years", "2-3 years",
                                            "3-4 years", "4-5 years", "5-10 years", 
                                            "10-20 years", "20 or more years"], ordered=True)
    data['Q15'] = data['Q15'].replace({'I do not use machine learning methods': '0 years'}).fillna('0 years')

    data['Q15'] = data['Q15'].astype(exp_cats)

    emp_cats = CategoricalDtype(categories=["0-49 employees", "50-249 employees", 
                                            "250-999 employees", "1000-9,999 employees",
                                            "10,000 or more employees"], ordered=True)
    data['Q20'] = data['Q20'].astype(emp_cats)

    size_cats = CategoricalDtype(categories=["0", "1-2", 
                                            "3-4", "5-9", "10-14", '15-19',
                                            "20+"], ordered=True)
    data['Q21'] = data['Q21'].astype(size_cats)

    data['Q22'] = data['Q22'].replace({'We are exploring ML methods (and may one day put a model into production)': 'Exploring', 
                                       'No (we do not use ML methods)': "No / I don't know", 'I do not know': "No / I don't know",
                                      "We have well established ML methods (i.e., models in production for more than 2 years)": "In prod. for more than 2 years", 
                                       "We recently started using ML methods (i.e., models in production for less than 2 years)": "In prod. for less than 2 years", 
                                       "We use ML methods for generating insights (but do not put working models into production)": 'Insights'})
    ml_cat = CategoricalDtype(categories=["No / I don't know", 'Exploring', 
                                            'Insights', "In prod. for less than 2 years", 
                                          "In prod. for more than 2 years"], ordered=True)
    data['Q22'] = data['Q22'].astype(ml_cat)
    return data

data = total_question(data, 'Q7')
data = total_question(data, 'Q16')
data = total_question(data, 'Q17')
data = total_question(data, 'Q18')
data = total_question(data, 'Q19')

df = data.copy()

data.loc[~data.Q2.isin(['Man', 'Woman']), 'Q2'] = 'Other'  # Will be analyzed separately

data = clean_data(data)
df = clean_data(df)


Every year, Kaggle users participate in a survey about their experience on the platform, their knowledge, their socio-economic status. With the growth of the community, this survey is becoming more and more a window over the world of Data Science and Machine Learning.

Here a quick overview of the survey audience

In [ ]:
fig = plt.figure(figsize=(15, 14), facecolor='#292525')
fig.subplots_adjust(top=0.94)
fig.suptitle('Kaggle 2020 Survey Overview', fontsize=18, color='w')

gs = GridSpec(3, 9, figure=fig)

ax0 = fig.add_subplot(gs[0, :4])
ax1 = fig.add_subplot(gs[0, 4:])
ax2 = fig.add_subplot(gs[1, :3])
ax3 = fig.add_subplot(gs[1, 4:])
ax4 = fig.add_subplot(gs[2, 1:8])

data.Q2.value_counts(normalize=True).plot(kind='pie', ax=ax0, 
                                          colors=['#C3C92E', '#C93D2E', '#41DA5B'], 
                                          textprops=dict(color="w"))
ax0.set_title('Gender', color='w', fontsize=14)
ax0.set_ylabel('')

data.Q1.value_counts().sort_index().plot(kind='bar', ax=ax1, color='#15E498')
ax1 = plot_frame(ax1)
plt.draw()
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.set_title('Age', color='w', fontsize=14)

data.continent.value_counts().plot(kind='bar', ax=ax2, color='#B8FD6E')
ax2 = plot_frame(ax2)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.set_title('Continent', color='w', fontsize=14)

data.Q24_num.hist(bins=20, ax=ax3, grid=False)
ax3 = plot_frame(ax3)
ax3.set_title('Salary', color='w', fontsize=14)

data.Q4.value_counts().sort_index().plot(kind='bar', ax=ax4, color='#f98e1d')
ax4 = plot_frame(ax4)
ax4.set_xticklabels(ax4.get_xticklabels(), rotation=30)
ax4.set_title('Education Level', color='w', fontsize=14)

plt.show()

In summary, it is a vastly male-dominated population of young age and with a university education. The gender unbalance is particularly striking, with 78% of the respondents identifying as Men, 19% as Women, and about 2% that either does not declare their gender, self-describe, or identifies as non-binary (we unfairly call this group generically as *Other*, mainly for lack of ideas of the author about how to properly display results with such low numerical representation). 

While a survey that runs over virtually the entire world with about 20000 respondents is not expected to be necessarily representative of the gender distribution of the population (thus approximatively 50/50), such unbalance raises questions on how valid any analysis of this data can be. In particular, it is very likely that any pattern one can find in the data will be representative of relations that are true for the majority population (in this case, Men).

Therefore, this notebook will focus instead on the differences between the 3 gender options presented above. The story the data will tell us is that **users that identify as Women are systematically paid less than the ones identifying as Men**. We will thus explore several aspects of the professional life of our respondents to try to make sense of this result, such as the size of the company, or years of experience. Unfortunately (for the health of our society), the gender gap will often look not justifiable by the available data.

The survey allowed for a more complete set of answers to the Gender question that we have grouped into a generic gender *Other* for simplicity. If we want to zoom in to this set of respondents we see that most of them prefer not to declare their gender.

In [ ]:
tmp = df[~df.Q2.isin(['Man', 'Woman'])]

fig = plt.figure(figsize=(15, 14), facecolor='#292525')
fig.subplots_adjust(top=0.94)
fig.suptitle('Gender "Other" Overview', fontsize=18, color='w')

gs = GridSpec(3, 9, figure=fig)

ax0 = fig.add_subplot(gs[0, :4])
ax1 = fig.add_subplot(gs[0, 4:])
ax2 = fig.add_subplot(gs[1, :3])
ax3 = fig.add_subplot(gs[1, 4:])
ax4 = fig.add_subplot(gs[2, 1:8])

tmp.Q2.value_counts(normalize=True).plot(kind='pie', ax=ax0, 
                                          textprops=dict(color="w"))
ax0.set_title('Gender', color='w', fontsize=14)
ax0.set_ylabel('')

tmp.Q1.value_counts().sort_index().plot(kind='bar', ax=ax1, color='#15E498')
ax1 = plot_frame(ax1)
plt.draw()
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.set_title('Age', color='w', fontsize=14)

tmp.continent.value_counts().plot(kind='bar', ax=ax2, color='#B8FD6E')
ax2 = plot_frame(ax2)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.set_title('Continent', color='w', fontsize=14)

tmp.Q24_num.hist(bins=20, ax=ax3, grid=False)
ax3 = plot_frame(ax3)
ax3.set_title('Salary', color='w', fontsize=14)

tmp.Q4.value_counts().sort_index().plot(kind='bar', ax=ax4, color='#f98e1d')
ax4 = plot_frame(ax4)
ax4.set_xticklabels(ax4.get_xticklabels(), rotation=30)
ax4.set_title('Education Level', color='w', fontsize=14)

plt.show()

Irregardless of the gender categorization, we see that there are a few respondents (2.5%) declaring a very high salary (above 200k per year). For the purpose of this analysis, we will ignore them as they would lead to more extreme results.

*Note: the salary question was asked in terms of a range of values (e.g. 1000-1999)*. *In this notebook, we will convert such ranges into numbers to better display averages. For example, if a respondent answered 1000-1999, they will have assigned the middle value 1500. Also for this reason, excluding the high categories is advised as the steps between one result and the next one become too big.*

In [ ]:
 data = data[data.Q24_num < 176000].copy()

# A salary gap

The first factor that is expected to influence the salary of a person is geography. For this reason, we start by having a look at the mean salary in each continent.

*The dotted lines are either averages over the entire set of respondents or averages by gender*

In [ ]:
plot_means(data, 'Q24_num', 'continent', 'Salary Gap by Continent', 'Continent', 'Salary')

We see how the salaries are generically lower in Africa and Asia and how, worldwide, **Women have a lower salary on average**. This is particularly evident in absolute values in **Oceania** and in relative values in **Asia**. The Americas seem to be the exception. 

However, if we break it down to sub-regions of these continents, we see

In [ ]:
plot_means(data, 'Q24_num', 'sub_region', 'Salary Gap by Sub-Region', 'Sub-Region', 'Salary')

North America, the region where our respondents receive a higher salary, has a salary gap between Men and the other genders. Moreover, we notice visible differences in the mean salary between genders in Northern Europe, West Asia, and South Africa. **There is no region in the world where Men and Women are receiving the same salary on average** and it is always a higher salary for Men.

Zooming in to the 20 countries with most respondents, we finally get some examples of Countries where Men and Women are paid similarly, for example in Canada, Nigeria, and Indonesia.

In [ ]:
top_countries = data.Q3.value_counts().head(21).index.to_list()
top_countries = [c for c in top_countries if 'Other' not in c]

plot_means(data[data.Q3.isin(top_countries)], 'Q24_num', 'Q3', 'Salary Gap in the top20 Countries', 'Country', 'Salary')

Another aspect that could influence the salary is the level of education. Once again, we observe that **no matter the level of education, Women are paid less than Men**. The highest gap is for respondents with no formal education and the smallest one for the ones with a Bachelor's degree.

In [ ]:
plot_means(data, 'Q24_num', 'Q4', 'The role of the education level', 'Education', 'Salary')

As one would expect, salaries increase on average the older one gets and, in this case, the gap between genders is not as visible as in the previous cases.

In [ ]:
plot_means(data, 'Q24_num', 'Q1', 'Gender Gap by Age', 'Age', 'Salary')

# The role and type of company

The world of Data Science and Machine Learning is somewhat still very diverse in titles and job description. This is also an aspect that could influence the salary of our respondents.

We indeed see that Product Managers, Data Scientists, and Data Engineers are paid more on average in our sample. However, with the exception of Product Mangers where the gap is very small and even in favour of Women, **every job title seems to come with a salary gap between genders**.

In [ ]:
plot_means(data[data.Q24_num.notna()], 'Q24_num', 'Q5', 'The role of the Job Title', 'Job Title', 'Salary')

We can also see that bigger companies tend to pay more, as expected, but we also see how Men and Women are simply on 2 different curves of growth.

In [ ]:
plot_means(data[data.Q24_num.notna()], 'Q24_num', 'Q20', 'Small Companies have a large Salary Gap', 'Company Size', 'Salary')

The same applies if we look at the salary gap in relation with the size of the data science team.

In [ ]:
plot_means(data[data.Q24_num.notna()], 'Q24_num', 'Q21', 'Large Data Science teams have a larger Salary Gap', 'DS team Size', 'Salary')

We all know is that some companies call data science *create an AI and maintain it to solve business problems*, others interpret this field as *make a pivot table in a spreadsheet and put it on a PowerPoint* (arguably, this notebook would fall into this second category). Clearly, the salary levels expected for these 2 types of companies are very different. 

It is indeed the case, the *more* a company is using ML, the higher the average salary. As in all previous cases, the salary gap appears unaffected by the different purposes that ML has in the business.

In [ ]:
plot_means(data[data.Q24_num.notna()], 'Q24_num', 'Q22', 'Role of ML in the business', 'Use of ML', 'Salary')

# Programming Experience

After looking at geografical factors, business diversities, and education, it is time to focus on a more practical and personal aspect that could influence the salary of a person: coding experience.

In [ ]:
plot_means(data, 'Q24_num', 'Q6', 'Years of Coding Experience', 'Experience', 'Salary')

Here we see that the salary increase with experience and the gap seems to be negligible in the first couple of years of experience. However, starting from the third year of experience, the salary gap appears again.

One could then think that with more years of experience one has more chances of knowing new programming languages. The survey asked for knowledge of individual languages (such as Python or C++) so we can simply consider the total number of programming languages that an individual is familiar with.

In [ ]:
plot_means(data, 'Q7_total', 'Q6', 'With More years of Experience the users tend to know more Programming Languages', 'Experience', '# languages')

This result is interesting for a couple of reasons:

* we indeed see that with more experience come a higher number of programming languages known.
* we see that Men on average know more programming languages than women, but this changes a lot at different experience levels
* up until 5 years of coding experience there are very small differences between Men and Women, which is encouraging because it could mean that **the learning opportunities are equally spread across Genders at the beginning of their careers**

With learning opportunities, we do not necessarily refer to the education level, which appears to be of little influence on the number of programming languages one knows. However, we also notice that a **knowledge gap** appears for people with a Bachelor's or higher degree.

In [ ]:
plot_means(data, 'Q7_total', 'Q4', 'Role of the Education level in the # of known programming languages', 'Education', '# languages')

The knowledge gap is almost always present if we consider the age of the respondents but, interestingly, it changes sign for people older than 21 and starts increasing above 40.

In [ ]:
plot_means(data, 'Q7_total', 'Q1', 'Age plays a much smaller role', 'Age', '# languages')

Now that we have investigated the difference between genders in the knowledge of programming languages, we need to investigate if this number is relevant to explain the salary gap shown above.

**Learning a third programming language is on average bringing a higher salary for a Man, but not for a Woman.**

In [ ]:
plot_means(data, 'Q24_num', 'Q7_total', 'The more programming languages one knows, the bigger the gender gap', '# languages', 'Salary')

Even if we factor in the years of experience, we see that **the average difference in salaries between Men and Women is almost always in favour of the Men, no matter the experience or the number of languages they know.**

In [ ]:
tmp = data[['Q2', 'Q7_total', 'Q6', 'Q24_num']].copy()
tmp = tmp.groupby(['Q2', 'Q6', 'Q7_total'])['Q24_num'].mean().unstack(0).reset_index().dropna()
tmp['Difference'] = tmp['Man'] - tmp['Woman']

fig, ax = plt.subplots(2,3, figsize=(15, 8), facecolor='#292525')
fig.suptitle('Years of Experience and # of Programming Languages', fontsize=18, color='w')

i=0
j=0

for experience in tmp.Q6.unique():
    if experience == '0 years':
        continue
    plot_df = tmp[tmp.Q6 == experience]
    cols = np.where(plot_df.Difference > 0, '#C3C92E', '#C93D2E')
    plot_df.plot.scatter(x='Q7_total', y='Difference', s=100, ax=ax[i][j], color=cols)
    ax[i][j].set_title(f'{experience} of experience', color='w')
    ax[i][j].axhline(0, linestyle='--', color='w')
    ax[i][j] = plot_frame(ax[i][j])
    ax[i][j].set_ylabel('')
    ax[i][j].set_xlabel('')
    j += 1
    if j == 3:
        i += 1
        j = 0
        
ax[0][0].set_ylabel('Salary Difference')
ax[1][0].set_ylabel('Salary Difference')
ax[1][0].set_xlabel('# languages')
ax[1][1].set_xlabel('# languages')
ax[1][2].set_xlabel('# languages')    

plt.show()

# Machine Learning experience

At last, since we are a ML community too, we need to look into the experience with Machine Learning.

In [ ]:
plot_means(data, 'Q24_num', 'Q15', 'More years of experience in ML give a higher salary', 'Experience', 'Salary')

Once again, the higher the experience, the higher the salary and, once again, Women are systematically paid less.

In this case, we also see that, on the contrary of the programmin languages case, the knowledge of ML frameworks is different for Men and Women, with Men knowning on average more ML frameworks.

This knowledge gap between gender is not so evident when we consider the number of ML algorithms normally used at work

In [ ]:
plot_means(data, 'Q16_total', 'Q15', 'Women use fewer ML frameworks at every level of Experience', 'Experience', '# ML frameworks')

In [ ]:
plot_means(data, 'Q17_total', 'Q15', 'The gap is less evident when we look at the number of ML Algorithms normally used', 'Experience', '# ML algorithms')

Once again, we can check if this aspect influences salaries enough to explain the gender gap.

In [ ]:
plot_means(data, 'Q24_num', 'Q16_total', 'Knowing more ML frameworks is not necessarily bringing a higher salary', '# ML frameworks', 'Salary')

And the same hold if we consider the numeber of ML algorithms.

While having more knowledge in this field does not necessarily bring a higher salary, we see again that the difference in salary between Men and Women is always in favour of the Men at any level of experience and knowledge.

In [ ]:
tmp = data[['Q2', 'Q16_total', 'Q15', 'Q24_num']].copy()
tmp = tmp.groupby(['Q2', 'Q15', 'Q16_total'])['Q24_num'].mean().unstack(0).reset_index().dropna()
tmp['Difference'] = (tmp['Man'] - tmp['Woman']) / 100

fig, ax = plt.subplots(2,4, figsize=(16, 8), facecolor='#292525')
fig.suptitle('Years of Experience and # of ML Frameworks', fontsize=18, color='w')

i=0
j=0

for experience in tmp.Q15.unique():
    if experience == '0 years':
        continue
    plot_df = tmp[tmp.Q15 == experience]
    cols = np.where(plot_df.Difference > 0, '#C3C92E', '#C93D2E')
    plot_df.plot.scatter(x='Q16_total', y='Difference', s=100, ax=ax[i][j], color=cols)
    ax[i][j].set_title(f'{experience} of experience', color='w')
    ax[i][j].axhline(0, linestyle='--', color='w')
    ax[i][j] = plot_frame(ax[i][j])
    ax[i][j].set_ylabel('')
    ax[i][j].set_xlabel('')
    j += 1
    if j == 4:
        i += 1
        j = 0
        
ax[0][0].set_ylabel('Salary Difference (100s)')
ax[1][0].set_ylabel('Salary Difference (100s)')
ax[1][0].set_xlabel('# ML Frameworks')
ax[1][1].set_xlabel('# ML Frameworks')
ax[1][2].set_xlabel('# ML Frameworks')
ax[1][3].set_xlabel('# ML Frameworks')

plt.show()

# Final notes

The Kaggle survey is a window over the Kaggle community and, as a proxy, over the Data Science and Machine Learning community. This window shows a world where people identifying themselves as women are paid systematically less than those that identify as men. This discrimination seems to be unaffected by geography, years of experience, level of education, age, company size, job title,  programming knowledge, and Machine learning knowledge.

However you cut the data, by taking individuals identical under the above aspects, you will find that **the gender difference is enough to justify a salary difference**. This is a very saddening conclusion for me and a result that demands a systemic change. 

While the gap is always present, it does not translate into a difference in knowledge for younger respondents and/or people with less experience. I find this an encouraging result as it could mean that a change is happening and the right learning opportunities are given regardless of gender. However, it is worrisome how a knowledge gap is more evident among respondents with at least a Bachelor's degree.

This notebook was not created to bring solutions, as I have none. What I have is a set of questions

* Why is the Kaggle community so much skewed towards people identifying themselves as Men?
* What can the community do to fix this issue?
* How can we keep guaranteeing access to knowledge in the DS and ML field so that the trend of *knowledge equality* can be sustained for years to come?
* What can we do to make sure that people doing the same job are being paid equally?
* Is there something in our community or our job that is encouraging this discrimination?


At last, the original idea of this notebook was to conclude with a model that correctly fit the data to show that indeed Gender will play a role in determining the salary of a person, as an extra demonstration of the injustice we saw in the data. However, due to the gender unbalance early attempts created models that were performing decently in predicting the salary of Men but not of the other genders, which is a common situation when dealing with unbalance in the data. The idea was then dropped to focus more on the message rather than a display of model fairness. But this set of data (as well as the ones from previous editions) would provide a good playground for such studies.